In [22]:
# Import bibliotek
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from time import sleep

In [8]:
# funkcja wykorzystana w kodach:

def zmiana_na_float (cecha):
    """ 
    Funkcja do zmiany danych poszczegolnych cech na float z odcięciem jednostki.
      
    Parametry:
    ----------
    cecha (str): Liczba podana jako ciąg znaków 
  
    Wynik:
    ------
    float: Ciąg znaków zamieniony na float.
    
    Przykład:
    ---------
    zmiana_na_float('50,2 m2')->50.2
      
    """
    
    return float(cecha.rsplit(' ',1)[0].replace(' ','').replace(',','.'))     #rsplit-rozdzielam po spacji, ale tylko jednej spacji licząc od końca. Jeśli w stringu będzie kilka spacji, to podzieli tylko po tej ostatniej   

## Kod do pobrania szczegółów ogłoszeń (widocznych z poziomu strony głównej)

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam jedynie dane z pierwszej strony zwróconej przez Otodom. Są to dane widoczne z poziomu tej strony ( bez zagłębiania się w szczegóły, które można zobaczyć tylko po wejściu w konkretne oferty)
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom, jak również szczegółowe dane widoczne tylko po wejściu w konkretną ofertę 


In [23]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strony)
zawartosc = requests.get('https://www.otodom.pl/sprzedaz/mieszkanie/poznan/?search%5Bfilter_float_m%3Ato%5D=30&search%5Bfilter_enum_market%5D%5B0%5D=secondary&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1')

In [24]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [30]:
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert


for item in wszystkie_oferty_items: # dla każdej cechy, którą chcę wydobyć, szukam jej klasy na stronie Otodom
    naglowek=item.find(class_='text-nowrap hidden-xs').get_text()
    liczba_pokoi=item.find(class_="offer-item-rooms hidden-xs").get_text()
    powierzchnia=item.find(class_="hidden-xs offer-item-area").get_text()
    cena=item.find(class_="offer-item-price").get_text()
    cena_zlm2=item.find(class_="hidden-xs offer-item-price-per-m").get_text()
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        
    # poniższe cechy zamieniam na float wg.wczesniej napisanej funkcji
    liczba_pokoi=zmiana_na_float(liczba_pokoi)
    powierzchnia=zmiana_na_float(powierzchnia)
    cena=zmiana_na_float(cena.strip())
    cena_zlm2 =zmiana_na_float(cena_zlm2)

    print('tytuł: ', naglowek)
    print('liczba_pokoi: ', liczba_pokoi)
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zl_m2): ', cena_zlm2)
    print(adres_www, '\n')
    
    f.writerow([naglowek, liczba_pokoi,powierzchnia, cena, cena_zlm2, adres_www ]) ## zapisanie do csv

tytuł:  Mieszkanie na sprzedaż: Poznań, Łazarz
liczba_pokoi:  1.0
powierzchnia (m2):  29.74
cena (zł):  212882.6
cena (zl_m2):  7158.0
https://www.otodom.pl/oferta/kawalerka-w-centrum-lazarza-ID3Nw4Y.html#0c1ad58ef4 

tytuł:  Mieszkanie na sprzedaż: Poznań, Grunwald
liczba_pokoi:  1.0
powierzchnia (m2):  28.8
cena (zł):  184900.0
cena (zl_m2):  6420.0
https://www.otodom.pl/oferta/mieszkanie-28-80-m-poznan-ID3NyGA.html#0c1ad58ef4 

tytuł:  Mieszkanie na sprzedaż: Poznań, Łazarz
liczba_pokoi:  1.0
powierzchnia (m2):  29.68
cena (zł):  270000.0
cena (zl_m2):  9097.0
https://www.otodom.pl/oferta/lokal-mieszkalny-poznan-lazarz-ul-lukaszewicza-ID3PjS8.html#0c1ad58ef4 

tytuł:  Mieszkanie na sprzedaż: Poznań, Grunwald
liczba_pokoi:  1.0
powierzchnia (m2):  27.0
cena (zł):  199000.0
cena (zl_m2):  7370.0
https://www.otodom.pl/oferta/kapitalna-kawalerka-w-super-cenie-grunwald-ID3PgZV.html#0c1ad58ef4 

tytuł:  Mieszkanie na sprzedaż: Poznań, Grunwald
liczba_pokoi:  1.0
powierzchnia (m2):  27.0
c

In [29]:
## zapisanie do CSV
f = csv.writer(open('oferty_mieszkania.csv', 'w'))
f.writerow(['tytuł', 'liczba_pokoi', 'powierzchnia','cena', 'cena_zlm2', 'adres_www' ])

58

## Kod do pobrania szczegółów zawartych wewnątrz konkretnych ogłoszeń

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, powierzchni, rodzaju nieruchomości - w tym wypadku mieszkania, itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam dane z pierwszej strony zwróconej przez Otodom, przy czym są to dane szczegółowe widoczne po otwarciu konkretnych oferT
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom

In [12]:
# wyciągam pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strona)
zawartosc = requests.get('https://www.otodom.pl/sprzedaz/mieszkanie/poznan/?search%5Bfilter_float_m%3Ato%5D=30&search%5Bfilter_enum_market%5D%5B0%5D=secondary&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1')

In [13]:
zawartosc.status_code

200

In [14]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [ ]:
# kod do zaczytania szczegółów zawartych w ofertach (przy przeglądaniu strony www widoczne są tylko po wejściu w daną ofertę)
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

for item in wszystkie_oferty_items:
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        zawartosc2=requests.get(adres_www)
        soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
        szczegoly=soup2.find_all(class_='article-offer')
        for item in szczegoly:       
            adres=item.find(class_='address-lane').get_text()
            powierzchnia=item.find(class_='big').get_text()
            cena=item.find(class_="box-price-value").get_text()
            pietro=item.find(class_='param_floor_no').get_text()
            liczba_pokoi=item.find(class_='room-lane').get_text()
            opis=item.find(itemprop='description').get_text()
    
    
        powierzchnia=zmiana_na_float(powierzchnia)   
        cena=zmiana_na_float(cena.strip())
        cena_zl_m2=round(cena/powierzchnia,2)
        liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]
        
 
    print('adres: ', adres.strip())
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zł/m2): ', cena_zl_m2)
    print('pietro: ', pietro)
    print('liczba_pokoi: ', liczba_pokoi[0])
    print('opis: ', opis)
    print(adres_www, '\n')
    
    #f.writerow([adres, powierzchnia,cena, cena_zl_m2, pietro, liczba_pokoi[0], opis ]) ## zapisanie do csv
 

In [23]:
f = csv.writer(open('oferty_mieszkania.csv', 'w'))
f.writerow(['adres', 'powierzchnia (m2)', 'cena (zł)','Cena (zł/m2)','pietro', 'liczba_pokoi','opis' ])

73

## Kod do pobrania ofert z kilku stron 

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w poprzednich kodach czytałam jedynie dane widoczne na pierwszej stronie zwrócoej przez Otodom, teraz zaczytam dane z wielu stron. Zrobię to na 2 sposoby:
    - w pierwszym kodzie użytkownik sam będzie podawał, z ilu stron chce uzyskać dane ( np. Otodom zwróci 10 stron, a użytkownik uzna, że chce dane tylko z 2 stron). Poza tym użytkownik będzie mógł wybrać, z których stron chce otrzymać dane, wpisując linki do tych stron
    - w drugim kodzie automatycznie zaczytam dane z wszystkich stron zwróconych przez Otodom
- dodatkowo uporządkuję pobrane dane wyorzystując bibliotekę Pandas (do csv będą eksportowane uporządkowane dane)


### Kod 1- użytkownik podaje,z których stron chce pobrać dane

In [24]:
strony=[]
for strona in range(0,int(input("Wpisz, ile stron chcesz przejrzeć"))):
    strona=input(str("Wpisz pełen adres strony- cały link"))
    strony.append(strona)
    print('strony: ', strony)

Wpisz, ile stron chcesz przejrzeć2
Wpisz pełen adres strony- cały linkhttps://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1&page=2
strony:  ['https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1&page=2']
Wpisz pełen adres strony- cały linkhttps://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1&page=3
strony:  ['https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&sear

In [25]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
opisy=[]
    
for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
     # usunięcie niechcianych danych- w tym wypadku promowanych ogloszen, ktore ponownie pojwiają sie dalej w ofertach
    link_do_usuniecia = soup.find(class_='listing-title listing-title-promoted')
    link_do_usuniecia.decompose()
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                pietro=item.find(class_='param_floor_no').get_text()
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                sleep(1)

                powierzchnia=zmiana_na_float(powierzchnia)   
                cena=zmiana_na_float(cena.strip())
                cena_zl_m2=round(cena/powierzchnia,2)
                liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]


                print('adres: ', adres.strip())# strip rozdziela ciąg znaków i zwraca listę 
                print('powierzchnia (m2): ', powierzchnia)
                print('cena (zł): ', cena)
                print('cena (zł/m2): ', cena_zl_m2)
                print('pietro: ', pietro)
                print('liczba_pokoi: ', liczba_pokoi[0])
                print('opis: ', opis)
                print(adres_www, '\n')

                # zapisuję wyciągnięte dane do takiej postaci, abym mogła je dalej zapisać w Pandasie
                adresy.append(adres.strip())
                powierzchnie.append(powierzchnia)
                ceny.append(cena)
                ceny_zlm2.append(cena_zl_m2)
                pietra.append(pietro)
                pokoje.append(liczba_pokoi[0])
                opisy.append(opis)


adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, ul. Jawornicka
Poznań, Grunwald
powierzchnia (m2):  31.0
cena (zł):  227500.0
cena (zł/m2):  7338.71
pietro:  Piętro 5 (z 10)
liczba_pokoi:  1
opis:  
SPRZEDAŻ BEZPOŚREDNIA  Zapraszam do zakupu bardzo ładnego mieszkania na Grunwaldzie w Poznaniu , ul Jawornicka. Lokal 31 m2. W mieszkaniu duży pokój 16m2, oddzielna kuchnia, łazienka , przedpokój oraz duża garderoba jako osobne pomieszczenie. Do lokalu przynależy również piwnica. Mieszkanie po kapitalnym remoncie, ładnie i funkcjonalnie urządzone. Wymienione wszystkie instalacje na nowe: elektryczna, wodna, kanalizacji , nowe okna z roletami wewnętrznymi, kuchnia z nowoczesną zabudową wraz z całym osprzętem: indukcja, okap, zmywarka, lodówka- wszystkie sprzęty dobrej jakości firm: Teka, Bosch, Elektrolux, Liebherr. Wymienione podłogi, bardzo ładna, spora łazienka z przemyślanym miejscem na suszenie prania. Nowe drzwi, wewnętrzne i zewnętrzne antywłamaniowe. Ciekawym rozwiąz

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, ul. Marcelińska
Poznań, Grunwald
powierzchnia (m2):  35.0
cena (zł):  366000.0
cena (zł/m2):  10457.14
pietro:  Piętro 2 (z 8)
liczba_pokoi:  2
opis:  
Polecam ofertę mieszkania dwupokojowego 35m2, położonego w budynku Atanera w doskonałej lokalizacji przy ul. Marcelińskiej.
W Cenie mieszkania wyposażenie.
 Mieszkanie wyposażone w meble pokojowe i kuchenne ze sprzętem AGD . Urządzona łazienka. Duże szafy w zabudowie na korytarzu. Drzwi wejściowe antywłamaniowe firmy Dierre , wszystkie okna posiadają rolety. Widok z balkonu na dziedziniec, plac zabaw.
 
Sprzęt AGD : piekarnik , płyta indukcyjna Siemens , okap Ciarko , zmywarka Bosch , duża lodówka z zamrażarką Samsung.
 
Aneks kuchenny: meble na wysoki połysk, szuflady z samodomykaczem firmy Blum, na ścianie lacobel.
 
Łazienka : pojemna pralka Samsung Ecco Bubble ,  Panel szklany na grafiką , Szafki pod wymiar kolorystycznie dopasowane do płytek.  Duże lustro podświetlane

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, ul. Jeleniogórska 1/3
Poznań, Grunwald
powierzchnia (m2):  30.51
cena (zł):  250000.0
cena (zł/m2):  8194.03
pietro:  Piętro parter (z 4)
liczba_pokoi:  1
opis:  
Do sprzedania kawalerka z osobną kuchnią o powierzchni 30,51 m2, położone w Poznaniu przy ul. Jeleniogórskiej 1/3.
Nieruchomość buduje firma ARCHICOM w ramach inwestycji o nazwie OSIEDLE PRZY LESIE MARCELIN etap IV. Odbiór mieszkania przewidziany na październik 2019r., max. termin podpisania umowy przenoszącej własność - marzec 2020.
Obecnie wybudowane są już 4 kondygnacje.

Mieszkanie usytuowane na parterze budynku od strony dziedzińca, przynależy do niego taras.

Standard wykończenia deweloperski podwyższony - między innymi o zewnętrzne rolety antywłamaniowe, jako załącznik w postaci zdjęć dołączam szczegółowy opis poziomu wykończenia oraz użytych do budowy materiałów. Jednocześnie udostępniam plan sytuacyjny położenia budowanego osiedla oraz budynku.
Inwestyc

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Łazarz, Hetmańska
Poznań, Łazarz
powierzchnia (m2):  21.0
cena (zł):  168000.0
cena (zł/m2):  8000.0
pietro:  Piętro 6 (z 6)
liczba_pokoi:  1
opis:  
Polecam na sprzedaż przytulną kawalerkę na ul. Hetmańskiej.Mieszkanie składa się z jednego pokoju ,oddzielnej kuchni ,łazienki oraz piwnicy.Mieszkanie w doskonałej lokalizacji Poznania i dostępem do pełnej infrastruktury (15min pieszo do Starego Browaru.)Przed blokiem parking na szlaban dla mieszkańców.Blok posiada windę oraz bardzo blisko przystanek tramwajowy.Doskonała inwestycja pod wynajem.Opłaty za czynsz to 230 zł(ogrzewanie miejskie,woda,śmieci.Mieszkanie bez obciążeń.Wolne od zaraz.Cena: 168 000 zł - do negocjacjiZAPEWNIAMY BEZPŁATNĄ POMOC W UZYSKANIU NAJKORZYSTNIEJSZEGO KREDYTU NA RYNKUZapraszam na prezentację.Kontakt: (pokaż numer), (pokaż numer)Powyższy opis ma charakter informacyjny i nie stanowi oferty w rozumieniu przepisów KCAGENT PROWADZĄCY Alternatywa BisTelefon	(poka

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Junacka
Poznań, Grunwald
powierzchnia (m2):  32.15
cena (zł):  229000.0
cena (zł/m2):  7122.86
pietro:  Piętro 3 (z 4)
liczba_pokoi:  2
opis:  
NA SPRZEDAŻ KAWALERKA DWUPOKOJOWA NA GRUNWALDZIE OBOK OSIEDLE KOPERNIKATaka oferta się nie powtórzy:ZAMIESZKAJ lub ZAINWESTUJ, ZARABIAJ - WYNAJEM GWARANTOWANY!Kawalerka dwupokojowa o powierzchni 32,15 m2 na trzecim piętrze w niskim bloku na Grunwaldzie ul. Junacka, w sąsiedztwie osedla Kopernika. ATUTY:	•	super układ pomieszczeń;	•	bardzo jasne, słoneczne mieszkanie;	•	świetna lokalizacja, wszędzie blisko;	•	przynależna komórka lokatorska;STANDARD WYKOŃCZENIA:	•	mieszkanie do odświeżenia; 	•	okna PCV - wymienione.LOKALIZACJA:ul. Junacka, Poznań - Grunwald, woj. wielkopolskie 	•	Świetna lokalizacja - pozwala na szybkie dotarcie w różne części miasta Poznania. 	•	Blisko do Parku Wilsona,  Kasprowicza oraz Palmiarni Poznańskiej. 	•	Obok znajduje się szkoła, przedszkole, oraz punkty h

...

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Górczyn, ul. Sielska
Poznań, Górczyn
powierzchnia (m2):  33.7
cena (zł):  234215.0
cena (zł/m2):  6950.0
pietro:  Piętro 3 (z 4)
liczba_pokoi:  2
opis:  
Oferujemy w nowo powstającej inwestycji Sielska14, funkcjonalne 2 pokojowe mieszkanie, zlokalizowane na 3 piętrze o metrażu 33,7m2, na które składa się pokój dzienny z aneksem kuchennym (17,8m2), sypialnia (8,4m2), łazienka (4,1m2) i hol (3,4m2), oraz balkon (3,8m2). Wystawa okien północno-wschodnia, widok na spokojne przestronne podwórze.  Doskonała lokalizacja w spokojnej bocznej ulicy zapewnia komfort użytkowania mieszkania, a jednocześnie odległość tylko 100 metrów od przystanku tramwajowego na ulicy Głogowskiej gwarantuje doskonałą komunikację z całym miastem.  Sielska 14 to 4 piętrowy budynek mieszkalny z 2 lokalami usługowymi w parterze. W inwestycji znajdą się 22 mieszkania i 36 miejsc postojowych w hali garażowej, indywidualnych garażach i na zewnątrz. Część miejsc w hali

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald
Poznań, Grunwald
powierzchnia (m2):  25.84
cena (zł):  220700.0
cena (zł/m2):  8541.02
pietro:  Piętro 2 (z 3)
liczba_pokoi:  1
opis:  
Szanowni Państwo, jeśli szukają Państwo nietuzinkowego mieszkania zgodnego z Państwa stylem i charakterem - zapraszam do zapoznania się z naszą ofertą Mieszkań w nowej inwestycji na Grunwaldzie. W ofercie dostępne będą zarówno kawalerki, jak i mieszkania 2-, 3- i 4-pokojowe - o metrażach od 26 do 90 m². Do dyspozycji mieszkańców będą także miejsca parkingowe w podziemnej hali garażowej oraz rowerownia.- Kupując mieszkanie z rynku pierwotnego oszczędzasz około 5% jego wartości brak prowizji, brak 2% PCC.Opis:  mieszkanie 1-pokojowe o powierzchni 25,84 m2 mieszczące się na drugim piętrze w kameralnym budynku w kształcie litery "C".- Budowa rozpoczyna się w styczniu 2019r. - Koniec budowy lipiec 2020 r.kawalerka będzie składał się z:- pokoju dziennego z wydzielona częścią do spania oraz kuchn

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Grochowska 
Poznań, Grunwald
powierzchnia (m2):  27.0
cena (zł):  215000.0
cena (zł/m2):  7962.96
pietro:  Piętro 4 (z 4)
liczba_pokoi:  1
opis:  
Polecam na sprzedaż kawalerka po remoncie bez umeblowania. W cenę wliczone są pralka oraz lodówka. Mieszkanie znajduje się w dogodnej lokalizacji w pobliżu Wydziału Ogrodnictwa i Architektury Krajobrazu Uniwersytetu Przyrodniczego, Instytutu Psychologii UAM, Instytutu Filozofii UAM, Wydziału Nauk Społecznych UAM oraz Wydziału Studiów Edukacyjnych UAM. Po drugiej stronie ulicy mieści się siłownia otwarta 24/7, apteka, Żabka oraz Targowisko Świt, które właśnie kończy być odnawiane. Lokalizacja jest dobrze skomunikowana, dwie minuty od lokalu znajdują się przystanki autobusowe takich linii, jak 82, 91, 77, 63, 50, 59, 48, którymi bezpośrednio dojechać można w tak kluczowe miejsca, jak Ogrody, Poznań Główny, Rondo Kaponiera, Garbary, czy pentla na os. Sobieskiego. Po wliczeniu prąd

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Łazarz
Poznań, Łazarz
powierzchnia (m2):  30.09
cena (zł):  229000.0
cena (zł/m2):  7610.5
pietro:  Piętro 2 (z 2)
liczba_pokoi:  2
opis:  
Obniżka ceny!

SPRZEDAŻ BEZPOŚREDNIO PRZEZ WŁAŚCICIELA
Polecamy dwupokojowe mieszkanie o powierzchni 30,9 m2 mieszczącą się w kamienicy położonej w Poznaniu przy ul. Glogowskiej 45, bezpośrednio przy Parku Wilsona
Mieszkanie składa się z:•salonu połączonego z aneksem kuchennym (układ mieszkania pozwolił na umieszczenie rozkładanej sofy, szafki TV, wysokiego stołu jadalnego a także obszernej szafy na ubrania) •pokoju, gdzie może znaleźć się sypialnia lub pokój dziecięcy (obecnie nieumeblowany) -8 m2•oraz łazienki z prysznicem i miejscem na pralkę

Do mieszkania przynależy piwnica. Możliwość parkowania na wewnętrznym parkingu.
Lokal usytuowany jest na drugim piętrze w oficynie kamienicy. Podwórko bardzo zadbane z dużą ilością zieleni.

Mieszkanie świeżo po remoncie, wykonanym na podstawie projekt

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Smardzewska
Poznań, Grunwald
powierzchnia (m2):  25.84
cena (zł):  223000.0
cena (zł/m2):  8630.03
pietro:  Piętro 3 (z 4)
liczba_pokoi:  1
opis:  
0 % PROWIZJI DLA KUPUJACYCH PRZY OFERTACH DEWELOPERSKICH0% PODATKU PCCRYNEK PIERWOTNYCerisier Résidence to wyjątkowo kameralny, jedynie 3-piętrowy budynek położony w dzielnicy Grunwald – z dala od zgiełku miasta, w przyjemnym otoczeniu zieleni i wiśniowych drzew, do których odnosi się nazwa inwestycji. 100 drzewek, które zostaną tutaj posadzone, będzie tworzyć jedyną w swoim rodzaju zieloną enklawę. Wyższy standard wykończenia – reprezentacyjne hole wejściowe, portale drzwiowe, wycieraczki wpuszczone w podłogęDuże okna i głębokie balkony z balustradami ze szkła lub z płytami HPL w kolorze drewnaOgródki przydomowe o powierzchni do 200 m²Mała architektura – ławki z pergolami i stojaki na roweryPodświetlenie elewacji oraz terenu inwestycjiSystem kontroli dostępu, w tym wideodomof

### Porządkowanie  danych - Pandas

In [26]:
# data frame 'oferty' zawiera szczegóły pobrane ze strony Otodom. Dalej będę edytować dane w tabeli, aby były one przedstawione w bardziej przejrzysty sposob
oferty=pd.DataFrame({'adres':adresy,
                               'powierzchnia (m2)':powierzchnie,
                                'cena (zł)':ceny,
                                'Cena (zł/m2)':ceny_zlm2,
                                'pietro':pietra, 
                                'liczba_pokoi':pokoje,
                                'opis': opisy})

In [27]:
oferty.head()

,adres,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",31.00,227500.0,7338.71,Piętro 5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
1,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",31.61,237000.0,7497.63,Piętro 2 (z 2),1,\n----OFERTA SPRZEDAŻY----\n\nOferujemy Państw...
2,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",32.00,289000.0,9031.25,Piętro 4 (z 4),2,\nNIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na ...
3,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",31.00,227500.0,7338.71,Piętro 5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
4,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",35.00,366000.0,10457.14,Piętro 2 (z 8),2,\nPolecam ofertę mieszkania dwupokojowego 35m2...


In [31]:
# w kolumnie 'adres' mam string z danymi typu:rodzaj nieruchomosci, wojewodztwo, miasto, dzielnica itd.
# rozdzielam kolumnę 'adres' tak, aby każdy rodzaj danych znalazł się  w oddzielnej kolumnie. Jednocześnie zachowuję dane z pozostałych kolumn
mieszkania_oferty = pd.concat([oferty['adres'].apply(lambda x: pd.Series(str(x).split(','))),\
                     oferty['powierzchnia (m2)'],\
                     oferty['cena (zł)'],\
                     oferty['Cena (zł/m2)'],\
                     oferty['pietro'],\
                     oferty['liczba_pokoi'],\
                     oferty['opis']                
                    ],axis=1,ignore_index=True)
nazwy_kolumn = ['nieruchomosc', 'wojewodztwo', 'miasto', 'dzielnica', 'ulica', 'reszta_inf', 'powierzchnia (m2)','cena (zł)', 'Cena (zł/m2)', 'pietro', 'liczba_pokoi', 'opis']
mieszkania_oferty.columns=nazwy_kolumn

In [32]:
mieszkania_oferty.head()

,nieruchomosc,wojewodztwo,miasto,dzielnica,ulica,reszta_inf,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka\nPoznań,Grunwald,31.00,227500.0,7338.71,Piętro 5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
1,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald\nPoznań,Grunwald,NaN,31.61,237000.0,7497.63,Piętro 2 (z 2),1,\n----OFERTA SPRZEDAŻY----\n\nOferujemy Państw...
2,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,GRUNWALDZKA 38\nPoznań,Grunwald,32.00,289000.0,9031.25,Piętro 4 (z 4),2,\nNIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na ...
3,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka\nPoznań,Grunwald,31.00,227500.0,7338.71,Piętro 5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
4,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Marcelińska\nPoznań,Grunwald,35.00,366000.0,10457.14,Piętro 2 (z 8),2,\nPolecam ofertę mieszkania dwupokojowego 35m2...


In [34]:
# usuwam kolumnę 'reszta_inf', ponieważ powtarzają się w niej dane zawarte w poprzedzających kolumnach
mieszkania_oferty.drop(['reszta_inf'], axis=1, inplace=True)

In [35]:
#usuwam wyraz'Piętro' z kolumny 'pietro'

In [ ]:
mieszkania_oferty['pietro']=mieszkania_oferty['pietro'].apply(lambda x: x.replace('Piętro',''))

In [36]:
mieszkania_oferty.head()

,nieruchomosc,wojewodztwo,miasto,dzielnica,ulica,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka\nPoznań,31.00,227500.0,7338.71,5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
1,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald\nPoznań,Grunwald,31.61,237000.0,7497.63,2 (z 2),1,\n----OFERTA SPRZEDAŻY----\n\nOferujemy Państw...
2,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,GRUNWALDZKA 38\nPoznań,32.00,289000.0,9031.25,4 (z 4),2,\nNIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na ...
3,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka\nPoznań,31.00,227500.0,7338.71,5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
4,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Marcelińska\nPoznań,35.00,366000.0,10457.14,2 (z 8),2,\nPolecam ofertę mieszkania dwupokojowego 35m2...


In [37]:
# usuwam '\nPoznań' z kolumny 'ulica'
mieszkania_oferty['ulica']=mieszkania_oferty['ulica'].apply(lambda x: x.replace('\nPoznań',''))

In [38]:
# usuwam '\nPoznań' z kolumny 'dzielinca'
mieszkania_oferty['dzielnica']=mieszkania_oferty['dzielnica'].apply(lambda x: x.replace('\nPoznań',''))

In [39]:
mieszkania_oferty.head()

,nieruchomosc,wojewodztwo,miasto,dzielnica,ulica,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka,31.00,227500.0,7338.71,5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
1,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Grunwald,31.61,237000.0,7497.63,2 (z 2),1,\n----OFERTA SPRZEDAŻY----\n\nOferujemy Państw...
2,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,GRUNWALDZKA 38,32.00,289000.0,9031.25,4 (z 4),2,\nNIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na ...
3,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka,31.00,227500.0,7338.71,5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
4,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Marcelińska,35.00,366000.0,10457.14,2 (z 8),2,\nPolecam ofertę mieszkania dwupokojowego 35m2...


In [41]:
mieszkania_oferty.drop_duplicates(subset=['powierzchnia (m2)', 'cena (zł)','pietro','liczba_pokoi' ])

,nieruchomosc,wojewodztwo,miasto,dzielnica,ulica,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jawornicka,31.00,227500.0,7338.71,5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
1,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Grunwald,31.61,237000.0,7497.63,2 (z 2),1,\n----OFERTA SPRZEDAŻY----\n\nOferujemy Państw...
2,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,GRUNWALDZKA 38,32.00,289000.0,9031.25,4 (z 4),2,\nNIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na ...
4,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Marcelińska,35.00,366000.0,10457.14,2 (z 8),2,\nPolecam ofertę mieszkania dwupokojowego 35m2...
5,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Grunwald,34.60,255000.0,7369.94,1 (z 4),1,\nWyjątkowa oferta!Mam przyjemność zaprezentow...
6,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Nowotomyska,30.69,204500.0,6663.41,3 (z 3),1,"\nNa sprzedaż kawalerka o powierzchni 30,69m2 ..."
7,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Górczyn,Krauthofera,35.00,240000.0,6857.14,5 (z 5),1,\nPolecam na sprzedaż mieszkanie na Górczynie...
8,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Łazarz,Głogowska,30.00,212000.0,7066.67,2 (z 4),1,\nPolecam na sprzedaż kawalerkę w kamienicy p...
10,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,ul. Jeleniogórska 1/3,30.51,250000.0,8194.03,parter (z 4),1,\nDo sprzedania kawalerka z osobną kuchnią o p...
11,Mieszkanie na sprzedaż,wielkopolskie,Poznań,Grunwald,Raszyńska,35.00,266900.0,7625.71,3 (z 3),2,"\nWitam, mam na imię Agata i w biurze nierucho..."


In [42]:
mieszkania_oferty.to_csv(r'/home/kinga/python/web_scrapping/mieszkania_oferty_po_poprawkach.csv')

### Kod 2 - automatycznie  pobiera dane ze wszystkich stron zwróconych przez Otodom

In [52]:

strona='https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1'
strony=[strona]
page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')

cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager') # w tym jest klasa 'page-counter' i 'page_next'

przycisk_przewijania_stron=cala_strona.find_all(class_='pager-next')


for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_str=item.find(class_='current').get_text()
    print (liczba_str)




4


In [53]:
i=1
while i in range(0,int(liczba_str)):
        i+=1
    
        strony.append(strona +'&page=' + str(i))  # każda kolejna strona ma końcówkę '&page=(nr_str)' w stosunku do strony pierwszej, poczynając od '&page=2' 

In [51]:
print ('strony: ', strony)

strony:  ['https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1', 'https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1&page=2', 'https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1&page=3', 'https://www.otodom.pl/sprzedaz/mieszkanie/poznan/grunwald/?search%5Bfilter_float_m%3Ato%5D=35&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bdistrict_id%5D=79&search%5Bsubregion_id%5D=462&search%5Bcity_id%5D=1&page=4']


In [121]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
opisy=[]
    
for item in strony:
    page = requests.get(item)
    soup=BeautifulSoup(page.content, 'html.parser')
     # usunięcie niechcianych danych- w tym wypadku promowanych ogloszen, ktore ponownie pojwiają sie dalej w ofertach
    link_do_usuniecia = soup.find(class_='listing-title listing-title-promoted')
    link_do_usuniecia.decompose()
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram linka do każdej oferty
            page2=requests.get(adres_www)
            soup2=BeautifulSoup(page2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                pietro=item.find(class_='param_floor_no').get_text()
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                sleep(1)

                powierzchnia=zmiana_na_float(powierzchnia)   
                cena=zmiana_na_float(cena.strip())
                cena_zl_m2=round(cena/powierzchnia,2)
                liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]


                print('adres: ', adres.strip())
                print('powierzchnia (m2): ', powierzchnia)
                print('cena (zł): ', cena)
                print('cena (zł/m2): ', cena_zl_m2)
                print('pietro: ', pietro)
                print('liczba_pokoi: ', liczba_pokoi[0])
                print('opis: ', opis)
                print(adres_www, '\n')

                # zapisuję pobrane dane do takiej postaci, abym mogła je dalej przedstawić w Pandasie
                adresy.append(adres.strip())
                powierzchnie.append(powierzchnia)
                ceny.append(cena)
                ceny_zlm2.append(cena_zl_m2)
                pietra.append(pietro)
                pokoje.append(liczba_pokoi[0])
                opisy.append(opis)


adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Łazarz, rynek Łazarski
Poznań, Łazarz
powierzchnia (m2):  29.74
cena (zł):  212882.6
cena (zł/m2):  7158.12
pietro:  Piętro 4 (z 4)
liczba_pokoi:  1
opis:  
--- KAWALERKA W SERCU ŁAZARZA---Polecam na sprzedaż kawalerkę na poznańskim Łazarzu w klimatycznej kamienicy, w stanie deweloperskim podwyższonym - panele na podłogach, okna trzyszybowe PCV, drzwi wewnętrzne, drzwi przeciwpożarowe wejściowe, płytki w kuchni, parapety, przygotowanie pod zestaw podtynkowy w łazience.Mieszkanie jednopoziomowe o powierzchni: 29,74 m2 wraz z powierzchnią dodatkową (piwnica)- 7,06 m2 wliczona w cenę mieszkania, znajdujące się w części oficynowej kamienicy z roku 1928.Wnętrza jasne, bardzo przestrzenne z uwagi na duże okna, które dają wyśmienity dostęp do światła dziennego. Wystawa okien: wschódCZYNSZ ADMINISTRACYJNY: 2 zł/m2; FUNDUSZ REMONTOWY: 4,50 zł/m2Pomieszczenia ustawne, można je zaaranżować według własnych preferencji, posiadające duży potencj

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Łazarz, rynek Łazarski
Poznań, Łazarz
powierzchnia (m2):  29.74
cena (zł):  212882.6
cena (zł/m2):  7158.12
pietro:  Piętro 4 (z 4)
liczba_pokoi:  1
opis:  
--- KAWALERKA W SERCU ŁAZARZA---Polecam na sprzedaż kawalerkę na poznańskim Łazarzu w klimatycznej kamienicy, w stanie deweloperskim podwyższonym - panele na podłogach, okna trzyszybowe PCV, drzwi wewnętrzne, drzwi przeciwpożarowe wejściowe, płytki w kuchni, parapety, przygotowanie pod zestaw podtynkowy w łazience.Mieszkanie jednopoziomowe o powierzchni: 29,74 m2 wraz z powierzchnią dodatkową (piwnica)- 7,06 m2 wliczona w cenę mieszkania, znajdujące się w części oficynowej kamienicy z roku 1928.Wnętrza jasne, bardzo przestrzenne z uwagi na duże okna, które dają wyśmienity dostęp do światła dziennego. Wystawa okien: wschódCZYNSZ ADMINISTRACYJNY: 2 zł/m2; FUNDUSZ REMONTOWY: 4,50 zł/m2Pomieszczenia ustawne, można je zaaranżować według własnych preferencji, posiadające duży potencj

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Drużbackiej
Poznań, Grunwald
powierzchnia (m2):  28.0
cena (zł):  200000.0
cena (zł/m2):  7142.86
pietro:  Piętro 5 (z 5)
liczba_pokoi:  1
opis:  
Polecam na sprzedaż bardzo zgrabną kawalerkę o powierzchni ok. 26 m2,  położoną w niskim, ocieplonym budynku z cegły na ul. Drużbackiej - Grunwald, Łazarz. Do mieszkania przynależy piwnica.Mieszkanie położone jest  na 5 piętrze bloku z windą ,  od strony wewnętrznego dziedzińca , wystawa okien zachodnia.Składa się z oddzielnej kuchni , pokoju z dużą szafą typu komandor ( na podłodze mozaika parkietowa) i łazienki.Ogrzewanie miejskie, woda ogrzewana junkersem. Do lokalu przynależy piwnica.Czynsz wynosi ok. 350 zł.  Idealna lokalizacja, w sąsiedztwie Arena, Park Wilsona, przystanki MPK, Rynek Łazarski oraz cała niezbędna infrastruktura. Znakomite połączenie komunikacyjne z resztą miasta oraz dworcem PKP/PKS. W okolicy jest Park Kasprowicza z pływalnią letnią i boiskami sportowymi

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Bułgarska
Poznań, Grunwald
powierzchnia (m2):  27.0
cena (zł):  203000.0
cena (zł/m2):  7518.52
pietro:  Piętro 3 (z 4)
liczba_pokoi:  1
opis:  
Zapraszam do zapoznania się z ofertą sprzedaży mieszkania położonego na ul. Bułgarskiej w Poznaniu.Atrakcyjna lokalizacja zapewniająca ciszę i spokój, bliskość terenów spacerowych oraz dostęp do infrastruktury handlowo-usługowej.Mieszkanie o powierzchni 27 mkw położone jest na 3 piętrze w 4 piętrowym bloku. Mieszkanie składa się z pokoju, kuchni oraz łazienki z WC. W korytarzu szafa typu komandor.Do mieszkania przynależy również piwnica. Czynsz do spółdzielni obejmuje zaliczkę na ciepłą i zimną wodę, ogrzewanie, wywóz śmieci, fundusz remontowy. Dodatkowe opłaty to prąd i gaz.Mieszkanie stanowi odrębną własność.Powyższa oferta nie stanowi oferty handlowej w rozumieniu art. 66 § 1 kodeksu cywilnego oraz innych właściwych przepisów prawnych.Zapraszam na prezentację, agent prowadzący

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Jawornicka
Poznań, Grunwald
powierzchnia (m2):  31.0
cena (zł):  238900.0
cena (zł/m2):  7706.45
pietro:  Piętro 6 (z 11)
liczba_pokoi:  2
opis:  
Witam, mam na imię Agata i w biurze nieruchomości Platan Investments pomagam moim klientom w odnalezieniu wymarzonego mieszkania. Robię to sprawnie i konkretnie mając na uwadze ulubioną mantrę- "czas to pieniądz". Specjalizuje się sprzedażą mieszkań głównie na poznańskim Grunwaldzie.Serdecznie polecam na sprzedaż, przestronne mieszkanie dwu pokojowe 31 m2 , znajdującą się na 6 piętrze w 11-piętrowym, w wyremontowanym bloku (ocieplenie, nowa elewacja, wymienione piony wodno-kanalizacyjne). Mieszkanie umiejscowione jest przy ul. JawornickiejW okolicy wszystkie potrzebne nam usługi handlowo-rozrywkowe, niesamowity zielony teren znajdujący się wokół nas daję poczucie bezpieczeństwa oraz uświadamia, że to właśnie Grunwald jest centrum naszych potrzeb.W skład mieszkania wchodzi:- sal

...

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Grunwaldzka 38
Poznań, Grunwald
powierzchnia (m2):  32.0
cena (zł):  289000.0
cena (zł/m2):  9031.25
pietro:  Piętro 2 (z 3)
liczba_pokoi:  2
opis:  
LOKALIZACJA ->LOKALIZACJA ->LOKALIZACJA !!!
  Poznań Grunwald, ulica Grunwaldzka 38 !

Mieszkanie 32 mkw w skład którego wchodzą:
- 2 pokoje
- kuchnia w aneksie
- łazienka
- korytarz z dużą szafą
- przestronny balkon
- wyposażenie i meble w cenie !!!  Niesamowita lokalizacja przylagająca do City Parku, gdzie możesz wyjść i sie odprężyć po ciężkim dniu.
Jeżeli wolisz naturę i zieleń, to przechodzisz ulicę i jesteś w parku przy Arenie.
Stąd na Uniwersytet Medyczny dojdziesz w kilka minut pieszo.
Zakupy ? Biedronka pod ręką. Chcesz wymyślać ? Tu wszystko blisko, ładnie i przyjemnie ... i jeszcze niedaleko są linie tramwajowe we wszystkich możliwych kierunkach.
Chcecie żyć komfortowo, w wielu aspektach ?

Centrum miasta, restauracje i puby City Parku, hala Arena, Park Sowińskieg

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, GRUNWALDZKA 38
Poznań, Grunwald
powierzchnia (m2):  32.0
cena (zł):  289000.0
cena (zł/m2):  9031.25
pietro:  Piętro 4 (z 4)
liczba_pokoi:  2
opis:  
NIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na rynku nieruchomości. Skorzystaj z mojej wiedzy, doświadczenia a może to oferta jest dla Ciebie ?Sprzedaję:^ doskonałą lokalizację - Grunwald ulica Grunwaldzka 38^ 2 pokoje z dużym balkonem w nowym budownictwie Układ pomieszczeń:^ jasny pokój połączony z otwartą kuchnią ^ niewielka sypialnia^ ciąg komunikacyjny z pojemną szafę typu garderoba^ duża łazienka z wanną^ windaAtuty:^ mieszkanie inwestycyjne na którym zarabiasz od razu, może być sprzedane z najemcą lub bez^ mieszkanie gotowe nie wymagające nakładów ^ podłogi panel, kafle^ mieszkanie idealnie nadaje się dla pary, studenta, singla, biuro ( okna od ulicy Grunwaldzkiej )^ mieszkanie ładnie zaaranżowane^ lokalizacja wyjątkowa ze względu na bliskie sąsiedztwo City Parku, Uniwers

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, ul. Marcelińska
Poznań, Grunwald
powierzchnia (m2):  35.0
cena (zł):  366000.0
cena (zł/m2):  10457.14
pietro:  Piętro 2 (z 8)
liczba_pokoi:  2
opis:  
Polecam ofertę mieszkania dwupokojowego 35m2, położonego w budynku Atanera w doskonałej lokalizacji przy ul. Marcelińskiej.
W Cenie mieszkania wyposażenie.
 Mieszkanie wyposażone w meble pokojowe i kuchenne ze sprzętem AGD . Urządzona łazienka. Duże szafy w zabudowie na korytarzu. Drzwi wejściowe antywłamaniowe firmy Dierre , wszystkie okna posiadają rolety. Widok z balkonu na dziedziniec, plac zabaw.
 
Sprzęt AGD : piekarnik , płyta indukcyjna Siemens , okap Ciarko , zmywarka Bosch , duża lodówka z zamrażarką Samsung.
 
Aneks kuchenny: meble na wysoki połysk, szuflady z samodomykaczem firmy Blum, na ścianie lacobel.
 
Łazienka : pojemna pralka Samsung Ecco Bubble ,  Panel szklany na grafiką , Szafki pod wymiar kolorystycznie dopasowane do płytek.  Duże lustro podświetlane

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, Marcelińska
Poznań, Grunwald
powierzchnia (m2):  22.0
cena (zł):  200000.0
cena (zł/m2):  9090.91
pietro:  Piętro parter (z 4)
liczba_pokoi:  1
opis:  
Polecam do zakupu kawalerkę 22m2 położoną na Grunwaldzie przy ul. Marcelińskiej. Powierzchnia mieszkania: pokój ( 12,0m2 ), jasna kuchnia ( 5,7m2 ), łazienka z wc ( 2,9m2 ), korytarz ( 2,3m2).Lokal położony na parterze w niskim bloku. Budynek wybudowany w 1946r. Mieszkanie częściowo odnowione w 2017r. Na terenie Wspólnoty miejsca postojowe.Czynsz za mieszkanie (maj 2018): 173,75 PLN ( w tym woda)Ogrzewanie elektryczne.Lokalizacja:Linie tramwajowy: 1,6,13,15Linie autobusowy: 45,69 i 48,50,91Sklepy: Lewiatan, Żabka, BiedronkaSzpital,Szkoły,Poczta Polska.Wyposażenie w cenie.Forma własności: pełna własność.Cena: 200 000 zł. Do negocjacji. Zapraszam na prezentację.GWARANTUJEMY POMOC W UZYSKANIU NAJKORZYSTNIEJSZEGO KREDYTU NA RYNKU.Kontakt: (pokaż numer), (pokaż numer)Zapewniamy

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, ul. Wałbrzyska 7
Poznań, Grunwald
powierzchnia (m2):  34.97
cena (zł):  229053.5
cena (zł/m2):  6550.0
pietro:  Piętro 4 (z 8)
liczba_pokoi:  1
opis:  
Zielony Marcelin I etap
mieszkanie nr 407, 1 balkon
Tylko teraz oferta promocyjna! Zadzwoń do biura sprzedaży i dopytaj o szczegóły. Tel. (pokaż numer), (pokaż numer). 
https://www.otodom.pl/oferta/zielony-marcelin-poznan-grunwald-1-pok-m-407-ID3hgFq.html#7339129e0f 

adres:  Mieszkanie na sprzedaż, wielkopolskie, Poznań, Grunwald, ul. Wałbrzyska 7
Poznań, Grunwald
powierzchnia (m2):  35.0
cena (zł):  223808.0
cena (zł/m2):  6394.51
pietro:  Piętro 1 (z 8)
liczba_pokoi:  1
opis:  
Zielony Marcelin I etap
mieszkanie nr 107, 1 balkon
Tylko teraz oferta promocyjna! Zadzwoń do biura sprzedaży i dopytaj o szczegóły. Tel. (pokaż numer), (pokaż numer). 
https://www.otodom.pl/oferta/zielony-marcelin-poznan-grunwald-1-pok-m-107-ID3hg2C.html#7339129e0f 



In [124]:
oferty_wszystkie_str=pd.DataFrame({'adres':adresy,
                               'powierzchnia (m2)':powierzchnie,
                                'cena (zł)':ceny,
                                'Cena (zł/m2)':ceny_zlm2,
                                'pietro':pietra, 
                                'liczba_pokoi':pokoje,
                                'opis': opisy})

In [125]:
oferty_wszystkie_str

,adres,powierzchnia (m2),cena (zł),Cena (zł/m2),pietro,liczba_pokoi,opis
0,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",29.74,212882.6,7158.12,Piętro 4 (z 4),1,\n--- KAWALERKA W SERCU ŁAZARZA---Polecam na s...
1,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",31.00,227500.0,7338.71,Piętro 5 (z 10),1,\nSPRZEDAŻ BEZPOŚREDNIA Zapraszam do zakupu b...
2,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",32.00,289000.0,9031.25,Piętro 4 (z 4),2,\nNIERUCHOMOŚCI NR 1 Ewa Tomaszewska10 lat na ...
3,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",26.22,214700.0,8188.41,Piętro 1 (z 3),1,\n0% prowizji. Przy tej ofercie biuro nie pobi...
4,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",29.74,212882.6,7158.12,Piętro 4 (z 4),1,\n--- KAWALERKA W SERCU ŁAZARZA---Polecam na s...
5,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",27.00,231000.0,8555.56,Piętro 4 (z 4),1,"\nSprzedam bezpośrednio komfortową kawalerkę, ..."
6,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",35.00,225000.0,6428.57,Piętro parter (z 3),1,\nPolecam do sprzedaży mieszkanie 1 pokojowe -...
7,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",35.00,249000.0,7114.29,Piętro 1 (z 4),1,\nMam przyjemność zaprezentować Państwu przest...
8,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",27.00,198000.0,7333.33,Piętro parter,1,\nOferuję na sprzedaż zadbaną kawalerkę na par...
9,"Mieszkanie na sprzedaż, wielkopolskie, Poznań,...",28.00,200000.0,7142.86,Piętro 5 (z 5),1,\nPolecam na sprzedaż bardzo zgrabną kawalerkę...


In [126]:
oferty_wszystkie_str.to_csv(r'/home/kinga/python/web_scrapping/oferty_wszystkie.csv')